In [21]:
import keras
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# Setting seed for reproducability
np.random.seed(1234)  
PYTHONHASHSEED = 0
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, recall_score, precision_score
from keras import models
from keras.wrappers.scikit_learn import KerasRegressor
from keras.models import Sequential,model_from_json
from keras.layers import Dense, Dropout, LSTM, Activation
%matplotlib inline
from decimal import Decimal
import warnings
warnings.filterwarnings("ignore")
import pickle
from sklearn.preprocessing import OneHotEncoder,LabelEncoder,Normalizer, MinMaxScaler
from sklearn.cross_validation import cross_val_score
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)
from sklearn.metrics import mean_absolute_error
from sklearn import ensemble
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
import xgboost as xgb

In [22]:
# df=pd.read_csv('C:\\Users\\lengada1\\NCSU\\DC_ten_skus.csv')
df=pd.read_csv('./DC_ten_skus.csv')
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.weekday_name
day_dummy=pd.get_dummies(df.Day)
df=pd.concat([df,day_dummy],axis=1)
df.drop(['Day','Date'],inplace=True,axis=1)

y=df['Sales']

In [23]:
df.head()
df.shape

(942, 16)

In [24]:
X=df.drop(['id','DayOfWeek','Customers','Sales'],inplace=False,axis=1)
X.head()

,Open,Promo,SchoolHoliday,Year,Month,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,0,0,10,2013,1,0,0,0,0,0,1,0
1,10,0,10,2013,1,0,0,0,0,0,0,1
2,10,0,8,2013,1,0,0,0,0,1,0,0
3,10,0,8,2013,1,1,0,0,0,0,0,0
4,10,0,1,2013,1,0,0,1,0,0,0,0


In [25]:
for obs in range(1,8):
    X["Sales_T"+str(obs)]=df['Sales'].shift(obs)


In [26]:
X['Mov_avg']=df['Sales'].rolling( window=7).mean().shift(1) 
y=y[7:]
X=X[7:]
y.reset_index(drop=True, inplace=True)
X.reset_index(drop=True, inplace=True)
X.head()

,Open,Promo,SchoolHoliday,Year,Month,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,Sales_T1,Sales_T2,Sales_T3,Sales_T4,Sales_T5,Sales_T6,Sales_T7,Mov_avg
0,10,10,1,2013,1,0,0,0,0,0,1,0,89391.0,0.0,43727.0,55606.0,52848.0,60436.0,0.0,43144.000000
1,10,10,1,2013,1,0,0,0,0,0,0,1,72552.0,89391.0,0.0,43727.0,55606.0,52848.0,60436.0,53508.571429
2,10,10,1,2013,1,0,0,0,0,1,0,0,64297.0,72552.0,89391.0,0.0,43727.0,55606.0,52848.0,54060.142857
3,10,10,1,2013,1,1,0,0,0,0,0,0,64112.0,64297.0,72552.0,89391.0,0.0,43727.0,55606.0,55669.285714
4,10,0,0,2013,1,0,0,1,0,0,0,0,65006.0,64112.0,64297.0,72552.0,89391.0,0.0,43727.0,57012.142857


In [27]:
std = preprocessing.StandardScaler().fit(X)
X = std.transform(X)
len(X)

935

In [28]:
cv=2

RF=RandomForestRegressor( n_estimators=1000,max_depth=10,max_features=14);
rf_models=[];
rf_models=(cross_val_score(RF,X,y,cv=cv, scoring ='mean_absolute_error').mean().round(0) )


GB=ensemble.GradientBoostingRegressor(n_estimators= 1000,max_depth= 8, max_features=14,
          learning_rate= 0.01, loss= 'ls');
gb_models=[];
gb_models=(cross_val_score(GB,X,y,cv=cv, scoring ='mean_absolute_error').mean().round(0)  )
   

XG=xgb.XGBRegressor(n_estimators= 900,learning_rate =0.01, max_depth=6,gamma=2)
xg_models=[]
xg_models=(cross_val_score(XG,X,y,cv=cv, scoring ='mean_absolute_error').mean().round(0)  )

In [29]:
def create_MLP():
    mlp=models.Sequential()
    mlp.add(Dense(20, input_dim=X.shape[1], activation='relu'))
    #mlp.add(Dense(20, activation='relu'))
    mlp.add(Dense(30, activation='relu'))
    mlp.add(Dense(10, activation='relu'))    
    mlp.add(Dense(1, activation='relu'))
    mlp.compile(loss='mae', optimizer='adam', metrics=['mae'])
    return mlp

MLP = KerasRegressor(build_fn=create_MLP,
                               epochs=300,
                               batch_size=20,
                               verbose=0)

mlp_models=[]
mlp_models=(cross_val_score(MLP,X,y,cv=cv).mean().round(0)  )

In [30]:
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process.kernels import RationalQuadratic
from sklearn.gaussian_process.kernels import ExpSineSquared
from sklearn.gaussian_process.kernels import Matern
from sklearn.gaussian_process import GaussianProcessRegressor

kernels = [    
    1.0* RBF() + 
    1.0* RBF() *
    1.0* ExpSineSquared(periodicity=7.0,length_scale_bounds=(1,10),periodicity_bounds=(7, 7) )   
]

n_restarts_optimizer=10

GPR=GaussianProcessRegressor(kernel=kernels[0], n_restarts_optimizer=n_restarts_optimizer)  

gpr_models=[]
gpr_models=(cross_val_score(GPR,X,y,cv=cv, scoring='mean_absolute_error').mean().round(0))

In [31]:
#Summary
mae=-pd.DataFrame(np.column_stack((mlp_models,rf_models,gb_models,xg_models,gpr_models)) )
mae=mae.rename(columns = {0:'MLP',1:'RF',2:'GB',3:'XG',4:"GPR"})
best=mae.idxmin(axis=1)
best.astype('category')
mae["Best"]=best
mae

,MLP,RF,GB,XG,GPR,Best
0,29346.0,3306.0,3220.0,3208.0,52117.0,XG


In [32]:
#Same models as above, but configured here to predict over entire X (935 days)
def RF_final(X, y):
    tree=RF
    tree.fit(X,y)
    pred=tree.predict(X)
    pred=pd.DataFrame(pred)
    pred.reset_index(drop=True, inplace=True)
    pred=pred.rename(columns = {0:'RF'})
    return (pred, tree)

def MLP_final(X,y):
    model = MLP
    model.fit(X,y)
    pred=model.predict(X)
    pred=pd.DataFrame(pred)
    pred.reset_index(drop=True, inplace=True)
    pred=pred.rename(columns = {0:'MLP'})
    return (pred, model)

def GB_final(X,y):
    model = GB
    model.fit(X,y)
    prediction=model.predict(X)
    pred=pd.DataFrame(prediction)
    pred.reset_index(drop=True, inplace=True)
    pred=pred.rename(columns = {0:'GB'})
    return (pred, model)

def XG_final(X,y):
    model=XG
    model.fit(X,y)
    prediction=model.predict(X)
    pred=pd.DataFrame(prediction)
    pred.reset_index(drop=True, inplace=True)
    pred=pred.rename(columns = {0:'XG'})
    return (pred, model)

In [33]:
#Using aggregated SKU sales data, the best performing model is selected based on K-fold. 
#Then best model is used to predict full data set 

bm_dict={}
for sku in range(0,1):        
        if  mae["Best"][0]=="RF":
            bm_dict[sku]=RF_final(X,y);
                    
        elif mae["Best"][0]=="NN":
            bm_dict[sku]=MLP_final(X,y);
                                
        elif mae["Best"][0]=="GB":
            bm_dict[sku]=GB_final(X,y); 
                        
        elif mae["Best"][0]=="XG":
            bm_dict[sku]=XG_final(X,y);

In [95]:
# Create Phase 2 data set which is comprised of both aggregate SKU data and forecasts
dc_pred=pd.DataFrame(bm_dict[0][0])
# sku_pred=pd.read_csv('C:\\Users\\lengada1\\NCSU\\prediction_skus.csv')
sku_pred=pd.read_csv('./prediction_skus.csv')

# Replace the SKU preds with SKU T1 data; Feel free to comment this out to resort back to SKU preds
sku_pred.loc[:,1:11]=pd.read_csv('./skus_t1.csv')     
sku_pred['Total'] = sku_pred.iloc[:,1:11].sum(axis=1)    # Replace SKU pred Total w/ sum of T1 lags

sku_pred=sku_pred.iloc[:,1:]
X2=pd.concat([sku_pred,dc_pred,pd.DataFrame(X)],axis=1)
sku_pred

,1,2,3,4,5,6,7,8,9,10,Total
0,7176.0,6775.0,12247.0,12112.0,6978.0,8591.0,12718.0,7416.0,7574.0,7804.0,89391.0
1,5580.0,6318.0,9800.0,10031.0,5718.0,7099.0,10073.0,6333.0,6206.0,5394.0,72552.0
2,5471.0,6763.0,8001.0,8857.0,5974.0,6749.0,7049.0,5038.0,4699.0,5696.0,64297.0
3,4892.0,5618.0,7772.0,9472.0,4999.0,6282.0,8234.0,6084.0,5340.0,5419.0,64112.0
4,4881.0,4810.0,9292.0,10512.0,5159.0,6829.0,7115.0,4997.0,5855.0,5556.0,65006.0
5,4952.0,2630.0,4455.0,9719.0,1760.0,3829.0,4236.0,2660.0,4657.0,4402.0,43300.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,4717.0,3995.0,4566.0,7794.0,4045.0,5908.0,6385.0,4128.0,4761.0,4685.0,50984.0
8,3900.0,4075.0,5041.0,7121.0,3916.0,5861.0,6780.0,4358.0,2724.0,4470.0,48246.0
9,4008.0,4198.0,4827.0,6537.0,3546.0,4454.0,5921.0,3508.0,3761.0,4640.0,45400.0


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.454589,1.266672,-0.272113,-1.103055,-1.499671,-0.409012,-0.407229,-0.407229,-0.407229,-0.409012,2.444916,-0.409012,1.370128,-1.931519,-0.315101,0.124641,0.024060,0.302992,-1.931749,-1.179223
1,0.454589,1.266672,-0.272113,-1.103055,-1.499671,-0.409012,-0.407229,-0.407229,-0.407229,-0.409012,-0.409012,2.444916,0.747023,1.370731,-1.930863,-0.314460,0.126098,0.021711,0.304175,0.161665
2,0.454589,1.266672,-0.272113,-1.103055,-1.499671,-0.409012,-0.407229,-0.407229,-0.407229,2.444916,-0.409012,-0.409012,0.441558,0.748671,1.372236,-1.930804,-0.313389,0.123948,0.023445,0.233023
3,0.454589,1.266672,-0.272113,-1.103055,-1.499671,2.444916,-0.407229,-0.407229,-0.407229,-0.409012,-0.409012,-0.409012,0.434712,0.443718,0.750016,1.373484,-1.931155,-0.316397,0.125482,0.441202
4,0.454589,-0.789471,-0.602536,-1.103055,-1.499671,-0.409012,-0.407229,2.455621,-0.407229,-0.409012,-0.409012,-0.409012,0.467794,0.436883,0.444984,0.751040,1.376040,-1.937323,-0.314000,0.614930
5,-2.235543,-0.789471,-0.602536,-1.103055,-1.499671,-0.409012,-0.407229,-0.407229,2.455621,-0.409012,-0.409012,-0.409012,-0.335408,0.469909,0.438148,0.445899,0.753048,1.376332,-1.931749,0.607039
6,0.454589,-0.789471,-0.602536,-1.103055,-1.499671,-0.409012,2.455621,-0.407229,-0.407229,-0.409012,-0.409012,-0.409012,-1.937666,-0.331946,0.471182,0.439060,0.447638,0.752123,1.375411,0.607039
7,0.454589,-0.789471,-0.602536,-1.103055,-1.499671,-0.409012,-0.407229,-0.407229,-0.407229,-0.409012,2.444916,-0.409012,-0.051072,-1.931519,-0.330879,0.472107,0.440794,0.446117,0.752426,-0.102790
8,0.454589,-0.789471,-0.602536,-1.103055,-1.499671,-0.409012,-0.407229,-0.407229,-0.407229,-0.409012,-0.409012,2.444916,-0.152388,-0.048086,-1.930863,-0.330244,0.473869,0.439259,0.447019,-0.552008
9,0.454589,-0.789471,-0.602536,-1.103055,-1.499671,-0.409012,-0.407229,-0.407229,-0.407229,2.444916,-0.409012,-0.409012,-0.257700,-0.149233,-0.046946,-1.930804,-0.329187,0.472399,0.440175,-0.901258


In [96]:
rng=range(0,33)
new_cols =  ['' + str(i) for i in rng]
X2.columns = new_cols[:32]
std2= preprocessing.StandardScaler().fit(X2)
X2 = std2.transform(X2)

In [97]:
def create_MLP2():
    mlp=models.Sequential()
    mlp.add(Dense(40, input_dim=X2.shape[1], activation='relu'))
    #mlp.add(Dense(20, activation='relu'))
    mlp.add(Dense(30, activation='relu'))
    mlp.add(Dense(10, activation='relu'))    
    mlp.add(Dense(1, activation='relu'))
    mlp.compile(loss='mae', optimizer='adam', metrics=['mae'])
    return mlp

MLP2 = KerasRegressor(build_fn=create_MLP2,
                               epochs=600,
                               batch_size=20,
                               verbose=0)

In [98]:
rf2_models=[];
rf2_models=(cross_val_score(RF,X2,y,cv=cv, scoring ='mean_absolute_error').mean().round(0) )

gb2_models=[];
gb2_models=(cross_val_score(GB,X2,y,cv=cv, scoring ='mean_absolute_error').mean().round(0)  )
   

xg2_models=[];
xg2_models=(cross_val_score(XG,X2,y,cv=cv, scoring ='mean_absolute_error').mean().round(0)  )

In [99]:

mlp2_models=[];
mlp2_models=(cross_val_score(MLP2,X2,y,cv=cv, scoring='mean_absolute_error').mean().round(0)  )

In [100]:
mae2=-pd.DataFrame(np.column_stack((mlp2_models,rf2_models,gb2_models,xg2_models)) )
mae2=mae2.rename(columns = {0:'MLP',1:'RF',2:'GB',3:'XG'})
mae2

,MLP,RF,GB,XG
0,1360.0,1420.0,1337.0,1048.0


In [101]:
#Using forecasted data and no empirical sales data, the best performaning model is selected based on Train/Test. 
## NOTE this improvement is not the true delta.  Only a final forecast between Stage 1 vs 2 will tell. 

best2=mae2.idxmin(axis=1)
best2.astype('category')
mae2["Best"]=best2
both_mae=pd.concat([mae,mae2],axis=0)
both_mae.reset_index(drop=True, inplace=True)

# Phase 1 & 2 DC Models compared based on Test set MAE scores. 
both_mae['min']=both_mae.min(axis=1)
both_mae['error reduction']= ((both_mae['min'][0]-both_mae['min'][1])/both_mae['min'][0] ).round(3)
both_mae

,Best,GB,GPR,MLP,RF,XG,min,error reduction
0,XG,3220.0,52117.0,29346.0,3306.0,3208.0,3208.0,0.673
1,XG,1337.0,NaN,1360.0,1420.0,1048.0,1048.0,0.673


In [80]:
bm2_dict={}
for sku in range(0,1):
        if  mae2["Best"][0]=="RF":
            bm2_dict[sku]=RF_final(X2,y);
                    
        elif mae2["Best"][0]=="NN":
            bm2_dict[sku]=MLP_final(X2,y);
                                
        elif mae2["Best"][0]=="GB":
            bm2_dict[sku]=GB_final(X2,y); 
                        
        elif mae2["Best"][0]=="XG":
            bm2_dict[sku]=XG_final(X2,y);    

In [81]:
tier_pred=pd.concat([bm_dict[0][0],bm2_dict[0][0],y],axis=1)
tier_pred.columns = ['DC_Phase1', 'DC_Phase2',"Sales"]
# tier_pred.to_csv('C:\\Users\\lengada1\\NCSU\\DC_tier_predictions.csv')
tier_pred.to_csv('./DC_tier_predictions.csv')
tier_pred.round()

,DC_Phase1,DC_Phase2,Sales
0,72276.0,72660.0,72552
1,64639.0,64302.0,64297
2,63063.0,63161.0,64112
3,65082.0,64750.0,65006
4,43302.0,43013.0,43300
5,-108.0,9.0,0
6,50399.0,50677.0,50984
7,48025.0,48281.0,48246
8,47117.0,46082.0,45400
9,47812.0,47342.0,46978


In [82]:
# END NOTEBOOK
bm_dict[0][1]

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=2, learning_rate=0.01, max_delta_step=0,
       max_depth=6, min_child_weight=1, missing=None, n_estimators=900,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [24]:
def save_model(model, filename):
    """
    Saves the model as a file. 
    - Keras models: saved as .json and .h5 files
    - scikit-learn models: saved as .pickle files
    """
    
    if type(model) is KerasRegressor:
        
        model_json = model.model.to_json()
        with open('{}.json'.format(filename), "w") as json_file:
            json_file.write(model_json)
        model.model.save_weights('{}.h5'.format(filename))        
    else:
        pickle_out = open('{}.pickle'.format(filename), "wb")
        pickle.dump(model, pickle_out)
        pickle_out.close()


# Save model to disk
filename="NCSU_DC"
model = bm_dict[0][1]
save_model(model, filename)


In [25]:
# Load model from disk
def load_pickled_model(path_to_model):
    loaded_model = pickle.load(open(path_to_model, 'rb'))
    
    return loaded_model

def load_keras_model(path_to_json, path_to_weights_h5):
    
    # load json and create model
    json_file = open(path_to_json, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)

    # load weights into new model
    loaded_model.load_weights(path_to_weights_h5)
    
    # wrap Sequential obj as a KerasRegressor to allow scikit-learn compatibility
    wrapped_loaded_model = KerasRegressor(build_fn=create_MLP, epochs=400, batch_size=20, verbose=0)
    
    return wrapped_loaded_model

def load_model(filename):
    """
    Loads a given file as a model
    - .pickle files: loaded as scikit-learn models
    - .json and .h5 files: loaded together to create Keras models
    """
    
    loaded_model = None
    
    try:
        loaded_model = load_pickled_model("{}.pickle".format(filename))
        
    except FileNotFoundError:
        loaded_model = load_keras_model("{}.json".format(filename), "{}.h5".format(filename))
        
    return loaded_model

loaded_model = load_model("NCSU_DC")

In [26]:
# Evaluate loaded model on test data
scores=[]
model_family=[]
    
# Identify the model family
if isinstance(loaded_model, RandomForestRegressor):
    model_family.append('RF')
    
elif isinstance(loaded_model, xgb.XGBRegressor):
    model_family.append('XG')   
    
elif isinstance(loaded_model, ensemble.GradientBoostingRegressor):
    model_family.append('GB')    
    
else:
    model_family.append('MLP')

# Obtain the cross validation score
score = -cross_val_score(loaded_model,X,y,cv=cv, scoring ='mean_absolute_error').mean().round(0)
scores.append(score)        

loaded_model_scores = pd.DataFrame({
    'CV Score':scores,
    'Family':model_family
})

loaded_model_scores

,CV Score,Family
0,3208.0,XG
